In [31]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import warnings
from fastai.vision import *
warnings.filterwarnings('ignore')

In [32]:
# chargement du model et de ses classes
learn = load_learner(path='models', file='model_fit_music.pkl')
classes = learn.data.classes

In [33]:
def predict_song(song_file):
    directory = pathlib.Path('data')
    for filename in directory.glob(song_file):
        print(filename)
        try:
            x, sample_rate = librosa.load(filename, offset=30, duration=30)
            n_fft = 1024
            hop_length = 256
            n_mels = 40
            fmin = 20
            fmax = sample_rate / 2

            mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate, n_fft=n_fft,
                                                            hop_length=hop_length,
                                                            n_mels=n_mels, power=2.0,
                                                            fmin=fmin, fmax=fmax)
            mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
            plt.imsave(pathlib.Path(directory).joinpath(
                filename.name[:-4]+".png"), mel_spec_db)
            
            print('coversion ok')
            
            predictions = learn.predict(open_image(pathlib.Path(directory).joinpath(
                filename.name[:-4]+".png")))
            probs_list = predictions[2].numpy()
           
            return {
                'category': classes[predictions[1].item()],
                'probs': {c: round(float(probs_list[i]), 5) for (i, c) in enumerate(classes)}
            }
        except Exception as e: print(e)

In [30]:
# en local seulement les .wav fonctionne avec librosa (probleme de config)
# normalement les .mp3 chez vous

predict_song("4.wav")

data\4.wav
coversion ok


{'category': 'nightcore',
 'probs': {'classic': 0.0,
  'jazz': 0.00101,
  'metal': 1e-05,
  'nightcore': 0.99898}}